In [ ]:
import pandas as pd
pd.options.mode.copy_on_write = True
from main_library import *
import pandas as pd, numpy as np
from functions_shared import cols_order, zipfile_ods, entreprise_cat_cleaning, cols_select
from step3_entities.ID_getSourceRef import get_source_ID
from config_path import PATH_CONNECT
projects = pd.read_pickle(f"{PATH_CLEAN}projects_current.pkl")
participation = pd.read_pickle(f"{PATH_CLEAN}participation_current.pkl")
countries = pd.read_pickle(f"{PATH_CLEAN}country_current.pkl")
entities_info = pd.read_pickle(f"{PATH_CLEAN}entities_info_current2.pkl")
calls = pd.read_csv(f"{PATH_CONNECT}calls.csv", sep=";", parse_dates=['call_deadline'])

# step4
entities_part = ent(participation, entities_info, projects)


# step5 - si nouvelle actualisation ou changement dans nomenclatures
h20, FP7, FP6, h20_p, FP7_p, FP6_p = framework_load()
h20 = h20.reindex(sorted(h20.columns), axis=1)

def select_cols_FP(FP):   
    cols_h=cols_select(FP, 'proj_entities')
    select=cols_h.loc[cols_h[FP].notna(), FP].unique()
    return select

def rename_cols_FP(FP):   
    cols_h=cols_select(FP, 'proj_entities')
    rename_map=cols_h[cols_h[FP].notna()].set_index(FP)['vars'].to_dict()
    return rename_map

def order_cols_FP(FP):   
    cols_h=cols_select(FP, 'proj_entities')
    # order_map=cols_h.loc[cols_h[FP].notna()].sort_values('order')
    order_map=cols_h.sort_values('order').vars.unique()
    return order_map

In [ ]:

def order_cols_FP(FP):   
    cols_h=cols_select(FP, 'proj_entities')
    # order_map=cols_h.loc[cols_h[FP].notna()].sort_values('order')
    order_map=cols_h.sort_values('order').vars.unique()
    return order_map

In [ ]:
entities_participation = entities_preparation(entities_part, h20)

In [ ]:
def entities_ods(FP, entities_participation):
    # ### entities pour ODS
    import math
    if FP=='horizon':
        filter_FP='Horizon Europe'
    elif FP=='h20':
        filter_FP='Horizon 2020'


    tmp=entities_participation[select_cols_FP(FP)].loc[(entities_participation.framework==filter_FP)]

    tmp=tmp.rename(columns=rename_cols_FP(FP))
    # cols_h=cols_select(FP, 'proj_entities')
    # select=cols_h[cols_h.horizon.notna()].horizon.unique()
    # rename_map=cols_h[cols_h.horizon.notna()].set_index('horizon')['vars'].to_dict()
    # tmp=entities_participation[select].rename(columns=rename_map)

    # rename_map=cols_h[cols_h[FP].notna()].set_index(FP)['vars'].to_dict()
    # tmp=df[select].rename(columns=rename_map)

    tmp.loc[tmp.entities_id_source=='ror', 'entities_id'] = tmp.loc[tmp.entities_id_source=='ror', 'entities_id'].str.replace("^R", "", regex=True)
    tmp.loc[tmp.entities_id_source=='pic', 'entities_id_source'] = 'ecorda pic'
    tmp.loc[tmp.entities_id_source=='identifiantAssociationUniteLegale', 'entities_id_source'] = 'rna'
    tmp.loc[(tmp.entities_id_source.isnull())&(tmp.entities_id.str.contains('gent', na=False)), 'entities_id_source'] = 'paysage'

    #     if i=='successful':
    act_liste = ['RIA', 'MSCA', 'IA', 'CSA', 'ERC', 'EIC']
    tmp = tmp.assign(action_group_code=tmp.action_id, action_group_name=tmp.action_name)
    tmp.loc[~tmp.action_id.isin(act_liste), 'action_group_code'] = 'ACT-OTHER'
    tmp.loc[~tmp.action_id.isin(act_liste), 'action_group_name'] = 'Others actions'

    # tmp.loc[tmp.thema_code.isin(['ERC','MSCA']), ['destination_code', 'destination_name_en']] = np.nan

    for i in ['abstract', 'free_keywords']:
        tmp[i] = tmp[i].str.replace('\\n|\\t|\\r|\\s+|^\\"', ' ', regex=True).str.strip()

    tmp['free_keywords'] = tmp['free_keywords'].str.lower()

    tmp.loc[(tmp.stage=='successful')&(tmp.status_code=='UNDER_PREPARATION'), 'abstract'] = np.nan

    l=[]
    for i in order_cols_FP(FP):
        if i in tmp.columns:
            l.append(i)
    tmp = tmp.reindex(columns=l)
    
    print(tmp.columns)
    # ATTENTION si changement de nom de vars -> la modifier aussi dans pcri_info_columns_order
    
    # for h in tmp.framework.unique():
    x = (tmp[(tmp.stage=='successful')]
            .drop(columns=['panel_regroupement_code', 'panel_code', 'erc_role', 'fund_ent_erc']))
    x.loc[x.thema_code.isin(['ERC','MSCA']), ['destination_code', 'destination_name_en']] = np.nan
    x = entreprise_cat_cleaning(x)
    chunk_size = int(math.ceil((x.shape[0] / 2)))
    i=0
    for start in range(0, x.shape[0], chunk_size):
        df_subset = x.iloc[start:start + chunk_size]
        i=i+1
        if FP=='h20':
            FP='h2020'
        zipfile_ods(df_subset, f"fr-esr-{FP}-projects-entities{i}")
 
    tmp1 = tmp.loc[(tmp.stage=='evaluated')].rename(columns={ 'number_involved':'numberofapplicants'})

    l=['country_name_mapping', 'country_association_name_en', 'country_name_en', 
            'country_code_mapping', 'pilier_name_fr', 'programme_code', 
            'operateur_num','operateur_lib', 'ror_category', 'paysage_category', 'country_association_name_en',
            'country_association_name_fr', 'thema_name_fr', 'destination_lib',
            'programme_name_fr', 'action_group_code', 'action_group_name', 
            'cordis_type_entity_name_en', 'cordis_type_entity_acro','cordis_type_entity_name_fr']
    del_i=[i for i in l if i in tmp1.columns]
    tmp1.drop(columns=del_i, inplace=True)

    # for h in tmp1.framework.unique():
    #     if h=='Horizon Europe':
    #         he='horizon'
    #         x = tmp1[(tmp1.framework==h)]
    #     else:
    if FP=='h20':
        FP='h2020'
        x = tmp1[tmp1.country_code=='FRA']
    else:
        x=tmp1
    chunk_size = int(math.ceil((x.shape[0] / 2)))
    i=0
    for start in range(0, x.shape[0], chunk_size):
        df_subset = x.iloc[start:start+chunk_size]
        i=i+1
        zipfile_ods(df_subset, f"fr-esr-{FP}-projects-entities-evaluated{i}")



In [ ]:
entities_ods('h20',entities_participation)
# entities_ods('horizon', entities_participation)